Hartree-Fock Method in TB-Model

Two basic thinking of Hartree-Fock:  
1\ The manybody state is a slater determinant.  
2\ Wick theory.  
For a manybody Hamiltonian 
    $$\hat{H} = \hat{c_i}^{\dagger}\hat{c_j}^{\dagger}\hat{c_k}\hat{c_m}$$
the expectation value of it is equal to the summation if all of the possible pairing cases, such as  
$$<\hat{c_i}^{\dagger}\hat{c_j}^{\dagger}><\hat{c_k}\hat{c_m}>$$
$$-<\hat{c_i}^{\dagger}\hat{c_k}><\hat{c_j}^{\dagger}\hat{c_m}>$$
$$<\hat{c_i}^{\dagger}\hat{c_m}><\hat{c_j}^{\dagger}\hat{c_k}>$$
If we adopt partical conservation, we can leave out cases like $<\hat{c_i}^{\dagger}\hat{c_j}^{\dagger}><\hat{c_k}\hat{c_m}>$.  
  
In Hartree-Fock Method, we ignore the scattering effect between different k points. Only in this way, can we get the quasi-particle band structures. We take the TB-model explained in this paper as an example: https://academic.oup.com/nsr/article/7/1/12/5549053?login=false.  
We only consider the nearest neighbor interaction:
$$\sum_{<i,j>} Un_in_j$$
The Hartree-Fock sample code is shown below.

In [ ]:
using LinearAlgebra
const t1 = 1
const t2 = 1
const t3 = 0
const eB =0
const eC = 0
const b1=[sqrt(3)/2,1/2,0]
const b2=[sqrt(3)/2,-1/2,0]
const b3=[0,0,1]
const V=dot(cross(b1,b2),b3)
const r1=deleteat!(2*pi*cross(b2,b3)/V,3) #array([3.62759873, 6.28318531]) # K(2/3,1/3) M (0.5,0)
const r2=deleteat!(2*pi*cross(b3,b1)/V,3) #array([3.62759873, -6.28318531])
# r1 and r2 are the reciprotical lattice
const d1 = [1/sqrt(3), 0]
const d2 = [-1/2, sqrt(3)/2]/sqrt(3)
const d3 = [-1/2, -sqrt(3)/2]/sqrt(3)
# d1 d2 d3 : B to C  and A to B
const tau1 = [-1/sqrt(3), 0]
const tau2 = [1/2, sqrt(3)/2]/sqrt(3)
const tau3 = [1/2, -sqrt(3)/2]/sqrt(3)
# tau1 tau2 tau3: A to C
const phi = pi/3

# iteration parameter
const λ = 1

function k_list(mesh=24)
    kx = Vector(range(0,1,mesh+1))
    ky = Vector(range(0,1,mesh+1))
    deleteat!(kx,mesh+1)
    deleteat!(ky,mesh+1)
    k_list = []
    for i in 1:mesh
        for j in 1:mesh
            if i==1 && j == 1
                k_list = [kx[i]*r1[1]+ky[j]*r2[1] kx[i]*r1[2]+ky[j]*r2[2]]
            else
                k_list = vcat(k_list,[kx[i]*r1[1]+ky[j]*r2[1] kx[i]*r1[2]+ky[j]*r2[2]])
            end
        end
    end
    return k_list
end
function Hk0(eA,kx,ky)
    Ham = zeros(ComplexF32,3,3)
    Ham[1,1] = eA
    Ham[2,2] = eB
    Ham[3,3] = eC
    Ham[2,1] = -t1*(exp(-1im*kx*d1[1]-1im*ky*d1[2])+exp(-1im*2*phi)*exp(-1im*kx*d2[1]-1im*ky*d2[2])+exp(1im*2*phi)*exp(-1im*kx*d3[1]-1im*ky*d3[2]))
    Ham[1,2] = conj(Ham[2,1])
    Ham[3,1] = -t2*(-exp(-1im*kx*tau1[1]-1im*ky*tau1[2])+exp(-1im*phi)*exp(-1im*kx*tau2[1]-1im*ky*tau2[2])+exp(1im*phi)*exp(-1im*kx*tau3[1]-1im*ky*tau3[2]))
    Ham[1,3] = conj(Ham[3,1])
    Ham[3,2] = -t3*(exp(-1im*kx*d1[1]-1im*ky*d1[2])+exp(-1im*kx*d2[1]-1im*ky*d2[2])+exp(-1im*kx*d3[1]-1im*ky*d3[2]))
    Ham[2,3] = conj(Ham[3,2])
    return Ham
end
function H0(eA,mesh=24)
    k = k_list(mesh)
    H0 = zeros(ComplexF32,mesh*mesh,3,3)
    Threads.@threads  for nk in 1:size(k,1)
        Ham = Hk0(eA,k[nk,1],k[nk,2])
        H0[nk,:,:] = Ham[:,:]
    end
    return H0
end
function Hartree(U,P,mesh = 24) #P[k,\alpha,\beta]
    V1 = U
    V2 = 0.9*U
    N = mesh*mesh
    k = k_list(mesh)
    Hartree = zeros(ComplexF32,mesh*mesh,3,3)
    Threads.@threads for n in 1:size(k,1)
        H_H = zeros(ComplexF32,3,3)
        H_H[1,1] = 3*V1/N*sum(P[:,2,2])+3*V1/N*sum(P[:,3,3])
        H_H[2,2] = 3*V1/N*sum(P[:,1,1])+3*V2/N*sum(P[:,3,3])
        H_H[3,3] = 3*V1/N*sum(P[:,1,1])+3*V2/N*sum(P[:,2,2])
        Hartree[n,:,:] = H_H[:,:]
    end
    return Hartree
end
function Fock(U,P,mesh = 24) #P[k,\alpha,\beta]
    V1 = U
    V2 = 0.9*U
    N = mesh*mesh
    k = k_list(mesh)
    Fock = zeros(ComplexF32,N,3,3)
    Threads.@threads for n = 1:N
        nk = k[n,:]
        H_F = zeros(ComplexF32,3,3)
        for m in 1:size(k,1)
            mk = k[m,:]
            H_F[2,1] -= V1/N*(exp(1im*dot(mk-nk,d1))+exp(1im*dot(mk-nk,d2))+exp(1im*dot(mk-nk,d3)))*P[m,2,1]
            H_F[3,1] -= V1/N*(exp(1im*dot(mk-nk,tau1))+exp(1im*dot(mk-nk,tau2))+exp(1im*dot(mk-nk,tau3)))*P[m,3,1]
            H_F[3,2] -= V2/N*(exp(1im*dot(mk-nk,d1))+exp(1im*dot(mk-nk,d2))+exp(1im*dot(mk-nk,d3)))*P[m,3,2]
        end
        H_F = H_F + H_F'
        Fock[n,:,:] = H_F[:,:]
    end
    return Fock
end
function t3_get(P,U,kmesh)
    N = size(P,1)
    corr = zeros(ComplexF64,N)
    Threads.@threads for n = 1:N
        corr[n]= exp(-1im*dot(d1,kmesh[n,:]))*P[n,2,3]
    end
    return sum(corr)/N*U
end
function Hartree_k(U,P,kx,ky) #P[k,\alpha,\beta]
    V1 = U
    V2 = 0.9*U
    N = size(P,1)
    H_H = zeros(ComplexF32,3,3)
    H_H[1,1] = 3*V1/N*sum(P[:,2,2])+3*V1/N*sum(P[:,3,3])
    H_H[2,2] = 3*V1/N*sum(P[:,1,1])+3*V2/N*sum(P[:,3,3])
    H_H[3,3] = 3*V1/N*sum(P[:,1,1])+3*V2/N*sum(P[:,2,2])
    return H_H
end

function Fock_k(U,P,kx,ky) #P[k,\alpha,\beta]
    V1 = U
    V2 = 0.9*U
    N = size(P,1)
    k = k_list(Int(sqrt(N)))
    H_F = zeros(ComplexF32,3,3)
    nk = [kx,ky]
    for m in 1:size(k,1)
        mk = k[m,:]
        H_F[2,1] -= V1/N*(exp(1im*dot(mk-nk,d1))+exp(1im*dot(mk-nk,d2))+exp(1im*dot(mk-nk,d3)))*P[m,2,1]
        H_F[3,1] -= V1/N*(exp(1im*dot(mk-nk,tau1))+exp(1im*dot(mk-nk,tau2))+exp(1im*dot(mk-nk,tau3)))*P[m,3,1]
        H_F[3,2] -= V2/N*(exp(1im*dot(mk-nk,d1))+exp(1im*dot(mk-nk,d2))+exp(1im*dot(mk-nk,d3)))*P[m,3,2]
    end
    H_F = H_F + H_F'
    return H_F
end

function cal_P(H,nu,mesh=24)
    P = zeros(ComplexF32,mesh*mesh,3,3)
    k = k_list(mesh)
    Threads.@threads for n in 1:size(k,1)
        Ham = zeros(ComplexF32,3,3)
        rho = zeros(ComplexF32,3,3)
        Ham[:,:] = H[n,:,:]
        vector = eigvecs(Ham)
        for f in 1:nu
            rho = rho+vector[:,f]*vector[:,f]'
        end
        P[n,:,:] = rho[:,:]
    end
    return P
end
function cal_E(H,nu,mesh=24)
    E = zeros(Float32,mesh*mesh,3)
    Threads.@threads for n in 1:size(H,1)
        Ham = zeros(ComplexF32,3,3)
        Ham[:,:] = H[n,:,:]
        E[n,:] = eigvals(Ham)
    end
    energy = 0
    for f in 1:nu
        energy += sum(E[:,f])
    end
    return energy
end
function cal_diff_P(P1,P0)
    diff = 0
    dP = P1-P0
    return sum(dP.*conj(dP))
end

function gen_random_P(mesh = 24)
    mesh = 24
    real = rand(mesh*mesh*9)
    cplx = rand(mesh*mesh*9)
    P = real + cplx*1im
    P = reshape(P,(mesh*mesh,3,3))
    for n in 1:mesh*mesh
        P[n,:,:] = (P[n,:,:]+P[n,:,:]')/2
        P[n,:,:] = P[n,:,:]/tr(P[n,:,:])
    end
    return P
end

function iter(eA,U,P0,nu,iter_max = 30)
    for it = 1:iter_max
        H_H = Hartree(U,P0)
        H_F = Fock(U,P0)
        H_0 = H0(eA)
        H_MF = H_H+H_F+H_0
        P1 = cal_P(H_MF,nu)
        diff_P = cal_diff_P(P1,P0)
        # println("diff_p = $diff_P \n")
        P0[:,:,:] = P1[:,:,:]*λ+P0[:,:,:]*(1-λ)
    end
    H_H = Hartree(U,P0)
    H_F = Fock(U,P0)
    H_0 = H0(eA)
    H_MF = H_H+H_F+H_0
    return P0,H_MF
end

let
    P = gen_random_P()
    gamma_gap = zeros(Float32,9,8)
    total_E = zeros(Float32,9,8)
    t3 = zeros(ComplexF64,9,8)
    kmesh = k_list()
    for U in 0.8:-0.1:0.1
        println(U)
        for eA in 9:-1:1
            P,H = iter(eA,U,P,1)
            total_E[eA,Int(round(U*10))] = cal_E(H,1)
            gamma_H_MF = Hk0(eA,0,0)+Hartree_k(U,P,0,0)+Fock_k(U,P,0,0)
            gamma_energy = eigvals(gamma_H_MF)
            gamma_gap[eA,Int(round(U*10))] = gamma_energy[2]-gamma_energy[1]
            t3[eA,Int(round(U*10))] = t3_get(P,U,kmesh)
        end
    end
    for eA in 9:-1:1
        println(eA)
        for U in 0.8:-0.1:0.1
            P,H = iter(eA,U,P,1)
            gamma_H_MF = Hk0(eA,0,0)+Hartree_k(U,P,0,0)+Fock_k(U,P,0,0)
            gamma_energy = eigvals(gamma_H_MF)
            if cal_E(H,1)<total_E[eA,Int(round(U*10))]
                total_E[eA,Int(round(U*10))] = cal_E(H,1)
                gamma_gap[eA,Int(round(U*10))] = gamma_energy[2]-gamma_energy[1]
                t3[eA,Int(round(U*10))] = t3_get(P,U,kmesh)
            end
        end
    end
    f = open("gamma_gap.txt","w")
    for neA = 1:9
        for nU = 1:8
            write(f,"$(gamma_gap[neA,nU])\t")
        end
        write(f,"\n")
    end
    close(f)

    f = open("t3.txt","w")
    for neA = 1:9
        for nU = 1:8
            write(f,"$(t3[neA,nU])\t")
        end
        write(f,"\n")
    end
    close(f)
end
